In [1]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
import sqlite3
sqlite3.register_adapter(np.int64, int)
import re
import math
from datetime import date
import matplotlib.pyplot as plt

# 展開所有dataframe columns
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# 不縮寫成科學記號
pd.set_option('display.float_format',lambda x : '%.2f' % x)

db = sqlite3.connect('../stock.db' , isolation_level=None)

In [2]:
QUARTERS = ['2023q1','2022q2','2022q3','2022q4']

In [3]:
# df
df_stock_id_name = pd.read_sql_query("select * from stockIdName",db)
daily = pd.read_sql_query("select * from daily",db)
daily = daily.sort_values(by=['date'])

df_financial_statement = pd.read_sql_query("select * from financialStatement",db)
df_financial_statement = df_financial_statement.fillna(0)
df_financial_statement.replace('--', 0, inplace=True)

In [4]:
ids = df_stock_id_name['id'].tolist()

## 月營收

In [5]:
sql=f"select * from monthlyRevenue order by date" 
df = pd.read_sql_query(sql, db)

In [6]:
# 用讀檔更新方式改寫
columns = ['id','name', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904', '201905', '201906', '201907', '201908', '201909', '201910', '201911', '201912', '202001', '202002', '202003', '202004', '202005', '202006', '202007', '202008', '202009', '202010', '202011', '202012', '202101', '202102', '202103', '202104', '202105', '202106', '202107', '202108', '202109', '202110', '202111', '202112', '202201', '202202', '202203', '202204','202205','202206', '202207', '202208','202209', '202210','202211','202212','202301','202302','202303','202304','202305','202306']
df_revenue = pd.DataFrame(columns=columns)
df_revenue

,id,name,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202301,202302,202303,202304,202305,202306


In [7]:
def getRowRevenue(id):
    a = df[df['id']==id][['date','name','revenue']]
    a['revenue'] = round(a['revenue']/100000, 2)
    b = a.T
    b.columns = b.iloc[0]
    b = b.drop(['date'])
    b['id'] = id
    b['name'] = b['201801']['name']
    b = b.drop(['name'])
    return b[columns]
for id in ids:
    try:
        row = getRowRevenue(id)
        df_revenue = df_revenue.append(row)
    except:
        continue


In [8]:
df_revenue.to_csv('revenue06.csv', index = False)
df_revenue.head()

,id,name,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202301,202302,202303,202304,202305,202306
revenue,1102,亞泥,67.70,34.09,63.95,71.88,76.98,75.50,70.21,71.80,77.04,76.58,76.26,65.49,79.06,31.66,83.19,84.35,84.36,75.37,78.13,72.03,73.95,70.16,80.01,81.15,52.10,28.51,50.77,67.52,71.83,67.52,67.99,66.83,72.57,74.33,79.24,83.21,74.36,34.50,70.07,78.31,76.89,68.96,64.78,71.93,78.31,94.55,89.86,94.04,74.21,41.01,81.60,87.10,80.00,77.76,78.65,70.69,69.94,76.01,83.06,83.41,48.14,62.41,81.53,71.69,72.39,66.71
revenue,1108,幸福,3.20,1.97,3.15,2.81,2.95,2.76,2.64,2.55,2.56,3.00,2.70,3.00,3.53,1.68,3.13,3.24,3.05,2.72,3.13,2.99,3.12,3.48,3.46,3.78,2.96,3.90,4.18,4.04,3.57,3.61,3.84,3.67,3.81,3.82,4.02,3.88,4.17,2.79,3.50,3.46,3.57,3.26,3.11,2.50,2.70,2.77,2.68,3.19,3.25,2.11,3.46,3.36,3.32,3.34,3.27,3.90,3.47,4.01,3.84,4.18,2.95,3.62,4.36,3.96,4.83,4.43
revenue,1109,信大,4.75,1.83,4.03,5.45,6.26,5.77,6.18,5.99,6.91,7.30,5.46,7.08,6.69,1.50,5.42,7.53,8.00,6.58,5.98,7.28,7.23,6.39,8.71,6.92,4.77,2.00,4.89,7.73,7.07,6.11,6.56,6.26,6.76,7.90,7.19,8.53,7.39,1.90,8.83,7.66,6.27,4.92,4.92,5.91,3.77,6.61,7.82,7.78,5.67,1.82,4.91,5.20,6.54,5.92,5.57,4.78,6.35,5.27,6.70,5.16,2.98,4.11,6.25,5.52,6.27,5.17
revenue,1110,東泥,1.53,0.97,1.62,1.18,1.43,1.19,1.57,0.99,1.28,1.78,1.39,1.84,1.86,0.83,1.48,1.18,1.32,1.14,1.15,1.21,1.22,1.43,1.52,1.51,1.26,1.15,1.46,1.46,1.06,1.30,1.25,0.99,1.43,1.44,1.50,1.61,1.38,1.03,1.46,1.44,1.42,2.62,1.59,1.09,1.40,1.51,1.61,1.85,1.67,0.97,1.79,1.14,0.90,1.15,1.34,1.77,1.57,1.98,1.98,1.68,1.38,1.74,2.16,2.19,2.14,1.82
revenue,1213,大飲,0.72,0.42,0.35,0.37,0.47,0.59,0.87,0.86,0.36,0.36,0.27,0.30,0.63,0.40,0.29,0.29,0.39,0.26,0.65,0.51,0.14,0.27,0.25,0.29,0.32,0.24,0.24,0.22,0.29,0.35,0.54,0.65,0.41,0.26,0.25,0.31,0.33,0.40,0.30,0.32,0.35,0.36,0.52,0.42,0.34,0.28,0.35,0.32,0.46,0.25,0.31,0.26,0.26,0.40,0.30,0.39,0.32,0.31,0.30,0.34,0.31,0.31,0.28,0.17,0.13,-0.03


## 每月收盤價

In [9]:
df_close = pd.DataFrame(columns=columns)
df_close

,id,name,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202301,202302,202303,202304,202305,202306


In [10]:
daily['yyyymm'] = daily['date'].apply(lambda x: str(x)[:6])

In [11]:
def getRowMonthClose(id):
    df = daily[daily['id']==id][['yyyymm','id','name','close']]
    name = df.head(1)['name'].iloc[0]
    df1 = round(df.groupby('yyyymm').mean().T, 0)
    df1 = df1.drop(['id'])
    df1['id'] = id
    df1['name'] = name
    try:
        return df1[columns]
    except:
        for col in columns:
            if col not in df1.columns:
                df1[col] = ''
        return df1[columns]

for id in ids:
    try:
        row = getRowMonthClose(id)
        df_close = df_close.append(row)
    except:
        continue

In [12]:
df_close.to_csv('monthlyClose06.csv', index = False)
df_close.head()

,id,name,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202301,202302,202303,202304,202305,202306
close,1101,台泥,38.00,37.00,37.00,39.00,43.00,44.00,42.00,42.00,40.00,38.00,35.00,35.00,36.00,38.00,39.00,42.00,42.00,45.00,45.00,40.00,40.00,40.00,41.00,42.00,44.00,43.00,40.00,41.00,43.00,43.00,45.00,44.00,42.00,41.00,42.00,43.00,42.00,41.00,44.00,50.00,51.00,51.00,52.00,49.00,50.00,49.00,48.00,47.00,47.00,47.00,49.00,47.00,43.00,40.00,40.00,39.00,38.00,32.00,32.00,34.00,35.00,37.00,37.00,38.00,38.00,38.00
close,1102,亞泥,30.00,28.00,29.00,30.00,33.00,34.00,37.00,43.00,39.00,37.00,33.00,34.00,35.00,38.00,39.00,41.00,43.00,47.00,46.00,42.00,44.00,43.00,44.00,46.00,48.00,46.00,40.00,42.00,44.00,44.00,44.00,42.00,42.00,41.00,43.00,43.00,42.00,42.00,45.00,49.00,50.00,51.00,52.00,46.00,46.00,45.00,44.00,43.00,44.00,45.00,47.00,47.00,45.00,44.00,44.00,42.00,43.00,38.00,40.00,41.00,42.00,43.00,43.00,44.00,44.00,44.00
close,1103,嘉泥,14.00,13.00,12.00,12.00,13.00,13.00,14.00,15.00,14.00,12.00,13.00,14.00,14.00,14.00,14.00,15.00,16.00,17.00,17.00,16.00,17.00,19.00,20.00,20.00,22.00,20.00,17.00,16.00,16.00,17.00,18.00,16.00,16.00,16.00,17.00,18.00,18.00,17.00,18.00,24.00,23.00,22.00,23.00,22.00,21.00,20.00,21.00,20.00,20.00,21.00,21.00,20.00,19.00,18.00,18.00,18.00,18.00,16.00,17.00,18.00,18.00,19.00,19.00,20.00,20.00,21.00
close,1104,環泥,23.00,22.00,22.00,22.00,22.00,21.00,20.00,20.00,20.00,19.00,19.00,19.00,19.00,19.00,20.00,20.00,20.00,20.00,19.00,19.00,19.00,19.00,19.00,19.00,19.00,18.00,16.00,17.00,18.00,18.00,18.00,18.00,20.00,20.00,22.00,23.00,22.00,21.00,22.00,25.00,23.00,23.00,23.00,22.00,21.00,21.00,21.00,22.00,22.00,22.00,21.00,21.00,21.00,22.00,22.00,22.00,22.00,20.00,21.00,22.00,23.00,24.00,25.00,27.00,29.00,30.00
close,1108,幸福,9.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,7.00,8.00,8.00,8.00,8.00,8.00,7.00,7.00,8.00,10.00,10.00,10.00,11.00,11.00,11.00,12.00,11.00,12.00,13.00,15.00,15.00,14.00,13.00,12.00,12.00,12.00,12.00,12.00,11.00,11.00,12.00,12.00,11.00,11.00,11.00,11.00,11.00,10.00,11.00,11.00,11.00,12.00,14.00,16.00,16.00,16.00


## EPS 

In [6]:
columns=['id','name','Q1','Q2','Q3','Q4']
df_eps = pd.DataFrame(columns=columns)
df_eps

# 客製化用QUARTERS排序
df_mapping_quarters = pd.DataFrame({
    'date': QUARTERS,
})
sort_mapping = df_mapping_quarters.reset_index().set_index('date')

# eps csv
def nearFourQuarterEPS(id):
    try:
        fid = df_financial_statement['id'] == id
        df = df_financial_statement[fid].tail(4)[['date','id','name','qeps']]
        df['order'] = df['date'].map(sort_mapping['index'])
        name = df['name'].iloc[0]
        df = df.sort_values('order')
        return [id, name] + df['qeps'].tolist()
    except:
        return np.nan
    

for id in ids:
    try:
        row = nearFourQuarterEPS(id)
        df_eps.loc[len(df_eps.index)] = row
    except:
        continue
        
df_eps.to_csv('EPS.csv', index = False)

In [39]:
dfq1 = df_financial_statement[df_financial_statement['date']=='2023q1']
sortedEps = dfq1[dfq1['eps']>0].sort_values(by=['eps'], ascending=False) # 1243/1777
sortedEps = sortedEps[['id','name','eps']]
sortedEps = sortedEps.reset_index(drop=True)
sortedEps.to_csv('2023q1_eps.csv')
sortedEps

,id,name,eps
0,3008,大立光,24.64
1,6669,緯穎,18.86
2,6472,保瑞,13.62
3,6488,環球晶,11.49
4,3533,嘉澤,10.96
5,2454,聯發科,10.64
6,3293,鈊象,10.24
7,2327,國巨,9.90
8,2207,和泰車,9.27
9,6409,旭隼,8.55


In [13]:
db.close()